## Modelo de recomendación ##

Procedemos a importar herramientas que utilizaremos, cargar el data set , y realizar acciones necesarias para que nuestro modelo recomiende de manera efectiva.

In [15]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

# Cargamos los datos 
data = pd.read_parquet("dataset_ok.parquet")

# Preprocesamiento

data['vote_average_scaled'] = MinMaxScaler().fit_transform(data[['vote_average']])
tfidf = TfidfVectorizer()
feature_matrix = tfidf.fit_transform(data['features'])

# Matriz de similitud
similarity_matrix = cosine_similarity(feature_matrix)

def recomendacion(titulo):
    """Recomienda películas similares a partir de un título.

    Args:
        titulo (str): Título de la película.

    Returns:
        list: Lista de títulos de películas recomendadas.
    """

    # Buscamos el índice de la película
    index = data[data['title'] == titulo].index[0]

    # Obtenemos las similitudes de la película
    similitudes = similarity_matrix[index]

    # Combinamos con similitud de puntuación
    similitudes = 0.5 * similitudes + 0.5 * (1 - abs(data['vote_average_scaled'] - data.loc[index, 'vote_average_scaled']))

    # Obtenemos los índices de las películas más similares
    top_indices = similitudes.argsort()[::-1][1:6]  # Excluimos la película original

    # Retornamos los títulos de las películas recomendadas
    return data.iloc[top_indices]['title'].tolist()



Hora de probar nuestro modelo!

In [20]:
recomendacion('shrek')


['the chronicles of narnia: the lion, the witch and the wardrobe',
 'cirque du soleil: worlds away',
 'post grad',
 'shrek 2',
 'tiny toon adventures: how i spent my vacation']